# CNN Classification Model -  Horses or Humans
- learning rate schedule
- early stopping callbacks

```
Step 1. Load libraries and data
Step 2. Data Preprocessing(ImageDataGenerator)
Step 3. Modeling
Step 4. Model Optimizer Setting
Step 5. Model Compile
Step 6. Callback Function Setting
     6-a. ModelCheckpoint
     6-b. EarlyStopping
Step 7. Model Fit 
Step 8. Model Evaluate & Save
```


## Step 1. Load libraries and data

In [3]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
train_path = './data/train_horses_or_humans'
valid_path = './data/valid_horses_or_humans/'

## Step 2. Data Preprocessing(ImageDataGenerator)

In [8]:
train_datagen = ImageDataGenerator(
    rescale = 1 / 255.0,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

valid_datagen = ImageDataGenerator(
    rescale = 1 / 255.0
)

In [11]:
train_data = train_datagen.flow_from_directory(
    train_path,
    target_size=(300,300),
    batch_size=64,
    class_mode='binary'
)

valid_data = valid_datagen.flow_from_directory(
    valid_path,
    target_size=(300,300),
    batch_size=64,
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


## Step 3. Modeling

In [12]:
model = Sequential([
    Conv2D(32, (3,3), input_shape=(300,300,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (2,2), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(256, (2,2), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

## Step 4. Model Optimizer Setting

In [14]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate = 2e-4,
    decay_steps = 200,
    decay_rate = 0.90
)
optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule)

## Step 5. Model Compile

In [15]:
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['acc'])

## Step 6. Callback Function Setting

### 6-a. ModelCheckpoint

In [16]:
cp_path = 'my_checkpoint_2.ckpt'
cp = ModelCheckpoint(
    filepath = cp_path,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    verbose=1
)

### 6-b. EarlyStopping

In [17]:
early_stopping_callback = EarlyStopping(monitor='val_loss', mode='min',min_delta=0.01, patience=100, restore_best_weights=True)

## Step 7. Model Fit

In [18]:
model.fit(
    train_data,
    validation_data = valid_data,
    epochs=30,
    callbacks=[cp, early_stopping_callback],
    verbose=1
)

Epoch 1/30
17/17 [==============================] - ETA: 0s - loss: 0.7233 - acc: 0.5414
Epoch 00001: val_loss improved from inf to 0.66272, saving model to my_checkpoint_2.ckpt
17/17 [==============================] - 48s 3s/step - loss: 0.7233 - acc: 0.5414 - val_loss: 0.6627 - val_acc: 0.5117
Epoch 2/30
17/17 [==============================] - ETA: 0s - loss: 0.5855 - acc: 0.7225
Epoch 00002: val_loss did not improve from 0.66272
17/17 [==============================] - 47s 3s/step - loss: 0.5855 - acc: 0.7225 - val_loss: 1.4080 - val_acc: 0.5000
Epoch 3/30
17/17 [==============================] - ETA: 0s - loss: 0.4819 - acc: 0.7595
Epoch 00003: val_loss did not improve from 0.66272
17/17 [==============================] - 52s 3s/step - loss: 0.4819 - acc: 0.7595 - val_loss: 1.2843 - val_acc: 0.5859
Epoch 4/30
17/17 [==============================] - ETA: 0s - loss: 0.4417 - acc: 0.7877
Epoch 00004: val_loss did not improve from 0.66272
17/17 [==============================] - 48s 

## Step 8. Model Evaluate & Save

In [19]:
model.load_weights(cp_path)
print(model.evaluate(valid_data))
print(model.summary())

4/4 [==============================] - 2s 424ms/step - loss: 0.6627 - acc: 0.5117
[0.6627198457717896, 0.51171875]
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 128)       32896     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 128)       0         
_______

In [20]:
model.save('./model/cnn_hoh_6627.h5')